In [ ]:
!pip install vaderSentiment textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install setuptools wheel
!pip install beautifulsoup4 lxml feedparser
!pip install git+https://github.com/clips/pattern.git


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.8 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=b838097af733006392476314ac90f5e8d10ef0e7304bb2a5bd4b02932e44dfb3
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k
  Cloning https://github.com/clips/pattern.git to /tmp/pip-req-build-z_etfw4b
  Running command git clone --filter=blob:none --quiet https://github.com/clips/pattern.git /tmp/pip-req-build-z_etfw4b
  Resolved https://github.com/clips/pattern.git to commit d25511f9ca7ed9356b801d8663b8b5168464e68f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from pattern.en import sentiment

# Sample text for sentiment analysis
text = "I love this product! It's amazing."
text1= "The service was slow and the food was cold."
text2="I really hate this. It’s terrible and disappointing."
text3="It is a day like any other."

# Vader Sentiment Analysis
vader_analyzer = SentimentIntensityAnalyzer()
vader_scores = vader_analyzer.polarity_scores(text)
vader_compound_score = vader_scores['compound']
vader_sentiment = 'positive' if vader_compound_score >= 0 else 'negative'

print(f"Vader Sentiment: {vader_sentiment} (Compound Score: {vader_compound_score})")

# TextBlob Sentiment Analysis
text_blob_analysis = TextBlob(text)
text_blob_polarity = text_blob_analysis.sentiment.polarity
text_blob_sentiment = 'positive' if text_blob_polarity > 0 else 'negative' if text_blob_polarity < 0 else 'neutral'

print(f"TextBlob Sentiment: {text_blob_sentiment} (Polarity: {text_blob_polarity})")

# Pattern Sentiment Analysis
pattern_sentiment = sentiment(text)
pattern_polarity = pattern_sentiment[0]
pattern_subjectivity = pattern_sentiment[1]
pattern_sentiment_label = 'positive' if pattern_polarity > 0 else 'negative' if pattern_polarity < 0 else 'neutral'

print(f"Pattern Sentiment: {pattern_sentiment_label} (Polarity: {pattern_polarity}, Subjectivity: {pattern_subjectivity})")

Vader Sentiment: positive (Compound Score: 0.8516)
TextBlob Sentiment: positive (Polarity: 0.6125)
Pattern Sentiment: positive (Polarity: 0.6125, Subjectivity: 0.75)


In [ ]:
import pandas as pd

df = pd.read_csv('/content/reviews.csv')

In [ ]:
display(df.head())

,Id,Review,Label
0,0,good and interesting,5
1,1,"This class is very helpful to me. Currently, I...",5
2,2,like!Prof and TAs are helpful and the discussi...,5
3,3,Easy to follow and includes a lot basic and im...,5
4,4,Really nice teacher!I could got the point eazl...,4


## Data cleaning


In [ ]:
import re

def clean_text(text):

    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.strip()
    return text

df['cleaned_review'] = df['Review'].apply(clean_text)

display(df[['Review', 'cleaned_review']].head())

,Review,cleaned_review
0,good and interesting,good and interesting
1,"This class is very helpful to me. Currently, I...",this class is very helpful to me currently im ...
2,like!Prof and TAs are helpful and the discussi...,likeprof and tas are helpful and the discussio...
3,Easy to follow and includes a lot basic and im...,easy to follow and includes a lot basic and im...
4,Really nice teacher!I could got the point eazl...,really nice teacheri could got the point eazli...


## Feature extraction


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cleaned_review'])
y = df['Label']

## Model training



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

## Model evaluation


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision (weighted): {precision}")
print(f"Recall (weighted): {recall}")
print(f"F1-score (weighted): {f1}")

Accuracy: 0.7773780601756681
Precision (weighted): 0.7365584978204681
Recall (weighted): 0.7773780601756681
F1-score (weighted): 0.747511381420489
